This colab notebook trains an augmented Wiki classifier discussed in section 6. 





In [ ]:
pip install transformers

     |████████████████████████████████| 2.3MB 5.2MB/s 
     |████████████████████████████████| 901kB 42.8MB/s 
     |████████████████████████████████| 3.3MB 49.9MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd 'Working Directory'

In [ ]:
import re
from tqdm import tqdm

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,precision_recall_fscore_support
import torch
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np 
from transformers import RobertaForSequenceClassification, RobertaTokenizerFast, Trainer, TrainingArguments
import torch

In [ ]:
from word_process import Word_Preprocessing
processor = Word_Preprocessing()

#Wiki Data

In [ ]:
toxicity_data = pd.read_csv('Wiki/wiki_lda_topics_lda_probabilities.csv')  #from https://arxiv.org/abs/2010.07414
test_toxic = toxicity_data[toxicity_data['split']=='test']
test_toxic['toxicity'] =test_toxic['toxicity'].apply(lambda x:1 if x else 0)
test_toxic = test_toxic.rename(columns={'comment':'text','toxicity':'label'})[['text','label']].reset_index(drop = True)
topic_categories={1:[0,1],
                  2:[2,7,8,9,12,14,16],
                  3:[3,4,5,6,10,11,13,15,17,18,19]}

train_toxic = toxicity_data[toxicity_data['split']=='train' ][toxicity_data['wiki_topic'].isin(topic_categories[1]+topic_categories[2])]
train_toxic['toxicity'] =train_toxic['toxicity'].apply(lambda x:1 if x else 0)
train_toxic = train_toxic.rename(columns={'comment':'text','toxicity':'label'})[['text','label']].reset_index(drop = True)

In [ ]:
train_toxic.groupby('label').count()

,text
label,
0,36121
1,7616


#Test Data

In [ ]:
#CH dataset 
racism_data = pd.read_csv('CH/annotations.csv',sep=',',encoding = "utf-8")

racism_data =processor.process_all(racism_data,'Text').rename(columns={'Text':'text','Label':'label'})[['text','label']].reset_index(drop = True)
racism_data['label'] = racism_data['label'].apply(lambda x : 1 if x in ['Hate','Non-Asian Aggression'] else 0).values.tolist()

racism_data.head()

 41%|████      | 950/2319 [00:00<00:00, 4570.82it/s]

Start eliminate url: : )


100%|██████████| 27/27 [00:00<00:00, 733.35it/s]

Start eliminate username: : )
Start eliminate symbol: : )
Start Final check: 
finished!!


,text,label
0,fucking piece of shit your whole community is...,1
1,i m not being funny but coronavirus in china ...,0
2,got on the victoria line today to seven sister...,1
3,it s appalling that the media amp libtards bit...,1
4,dude fuck the chinese man fuck em and if you t...,1


In [ ]:
#EA test set 
EA_test = pd.read_csv('Data/EA_test.csv')[['text','label']].reset_index(drop = True) 
EA_test['label'] =EA_test['label'].apply(lambda x: 1 if x==2 else 0)
EA_test.head()

,text,label
0,afraid chinapneumonia,0
1,rt hongkongers pls protect ourselves chinesep...,0
2,when china says 25 cases the ture number coul...,1
3,rt chinapneumonia is spreading to the world,0
4,based on the history china has the tendency ...,0


#Augmentation data

In [ ]:
EA_s_sorted = pd.read_csv('toxic_DoEs.csv').sort_values(by= ['toxic-DoE']).drop_duplicates(subset =['text'],keep='first')

In [ ]:
EA_s_sorted.groupby('label')['text'].count()

label
0    10646
1      911
2     3102
3     1143
Name: text, dtype: int64

In [ ]:
#1000 augmentation samples with lowest DoE
augmnetation =pd.DataFrame()
augmnetation['text'] = list(EA_s_sorted['text'].values)[:1000]#implicit_EA_dev_hate
EA_s_sorted['label'] = EA_s_sorted['label'].apply(lambda x:1 if x==2 else 0)
augmnetation['label'] = list(EA_s_sorted['label'].values)[:1000]#[1 for i in range(len(implicit_EA_dev_hate))]

In [ ]:
augmnetation.groupby('label')['text'].count()

label
0    866
1    134
Name: text, dtype: int64

In [ ]:

train_data = train_toxic.append(augmnetation)

In [ ]:
train_toxic.groupby('label')['text'].count(),train_data.groupby('label')['text'].count(),augmnetation.groupby('label')['text'].count()

(label
 0    36121
 1     7616
 Name: text, dtype: int64, label
 0    36987
 1     7750
 Name: text, dtype: int64, label
 0    866
 1    134
 Name: text, dtype: int64)

#train

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device('cpu')

print(device)

cuda:0


In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base',num_labels = len(train_data['label'].unique()))
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
model.num_labels 

2

In [ ]:
model

In [ ]:
class ToxicityDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_encodings = tokenizer(train_data['text'].values.tolist(), truncation=True, padding=True)

train_labels =train_data['label'].values.tolist()

train_dataset = ToxicityDataset(train_encodings, train_labels)



In [ ]:
test_toxic_encodings = tokenizer(test_toxic['text'].values.tolist(), truncation=True, padding=True)
test_toxic_labels = test_toxic['label'].values.tolist()
test_toxic_dataset = ToxicityDataset(test_toxic_encodings, test_toxic_labels)


test_racism_encodings = tokenizer(racism_data['text'].values.tolist(), truncation=True, padding=True)
test_racism_labels = racism_data['label'].values.tolist()
test_racism_dataset = ToxicityDataset(test_racism_encodings, test_racism_labels)

test_EA_encodings = tokenizer(EA_test['text'].values.tolist(), truncation=True, padding=True)
test_EA_labels = EA_test['label'].values.tolist()
test_EA_dataset = ToxicityDataset(test_EA_encodings, test_EA_labels)

In [ ]:
def compute_metrics(pred, average = 'macro'):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=average)
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds))
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
train_epoch = 3

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=train_epoch,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    )

In [ ]:
trainer.train()

#test on original data (wiki)

In [ ]:
toxic_preds = trainer.predict(test_toxic_dataset)

In [ ]:
compute_metrics(toxic_preds,average = 'micro')

#test on CH

In [ ]:
racism_preds = trainer.predict(test_racism_dataset)

In [ ]:
compute_metrics(racism_preds,average = 'micro')

#test on EA

In [ ]:
EA_preds = trainer.predict(test_EA_dataset)

In [ ]:
compute_metrics(EA_preds,average = 'micro')